## 라이브러리 로드

In [1]:
# 셀레니움 잘 깔렸는지 확인
!python --version

import selenium
print(selenium.__version__)

Python 3.12.10
4.39.0


In [2]:
import os
import requests
import pandas as pd
import sqlite3
import re
import sys

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import chromedriver_autoinstaller  # setup chrome options

import urllib.request
from urllib.request import urlretrieve
import time

import warnings
warnings.filterwarnings('ignore')

# 프로세스

In [3]:
import random

SAVE_PATH = 'G:\내 드라이브\01_Gnewsoft_Crawling\01_IEEE_TGRS_Restart'
TITLE = "IEEE Transactions on Geoscience and Remote Sensing" # 목표로 하는 저널명
START_PAGE = 232              # 재시작할 페이지 번호 (예: 1)
DOWNLOAD_WAIT_SECONDS = 270   # 대량 PDF 다운로드 대기 시간 (약 4분 30초)
PAGE_CHANGE_DELAY = 5        # 페이지 이동 후 안정화 대기 시간 (약 5초)
MAX_PAGE_VISITS = 50         # 안전장치: 최대 50페이지 처리
SET_YEAR = "2022"

# ===== Seat Limit 회피 설정 =====
SEAT_LIMIT_WAIT_SECONDS = 300  # Seat limit 감지 시 대기 시간 (5분)
MIN_RANDOM_DELAY = 3           # 최소 랜덤 지연 (초)
MAX_RANDOM_DELAY = 8           # 최대 랜덤 지연 (초)
MAX_SEAT_LIMIT_RETRIES = 5     # Seat limit 최대 재시도 횟수

# 기관 로그인

In [4]:
from selenium.webdriver.chrome.webdriver import WebDriver

import pandas as pd
import openpyxl
import os
import sys
import time
import json
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, TimeoutException, StaleElementReferenceException
from tqdm import tqdm
from selenium.webdriver.support.ui import WebDriverWait
import re

# Chrome 옵션 설정
options = webdriver.ChromeOptions()
options.add_experimental_option("detach", True)  # 브라우저가 스크립트 종료 후에도 유지되도록 설정
options.add_experimental_option("excludeSwitches", ["enable-logging"])
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_experimental_option("useAutomationExtension", False)
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-gpu")
options.page_load_strategy = 'eager'  # 페이지 로드 전략: eager (DOMContentLoaded까지만 대기)
options.set_capability('pageLoadStrategy', 'eager')

# 드라이버 초기화 (권한 문제 해결)
import subprocess
import shutil

def kill_chromedriver_processes():
    """실행 중인 ChromeDriver 프로세스 종료"""
    try:
        subprocess.run(['taskkill', '/F', '/IM', 'chromedriver.exe'], 
                      capture_output=True, check=False)
    except:
        pass

def setup_chromedriver():
    """ChromeDriver 설정 (권한 문제 해결)"""
    # 실행 중인 ChromeDriver 프로세스 종료
    kill_chromedriver_processes()
    time.sleep(1)
    
    # 방법 1: chromedriver_autoinstaller 사용 시도
    try:
        chromedriver_path = chromedriver_autoinstaller.install()
        print(f"  ✓ chromedriver_autoinstaller로 설치 완료: {chromedriver_path}")
        return chromedriver_path
    except Exception as e:
        print(f"  ⚠ chromedriver_autoinstaller 실패: {e}")
    
    # 방법 2: ChromeDriverManager 사용 (재시도 로직 포함)
    max_retries = 3
    for attempt in range(1, max_retries + 1):
        try:
            # .wdm 폴더의 잠긴 파일 정리 시도
            if attempt > 1:
                wdm_path = os.path.join(os.path.expanduser("~"), ".wdm")
                if os.path.exists(wdm_path):
                    try:
                        # 잠긴 파일이 있을 수 있는 경로
                        chromedriver_cache = os.path.join(wdm_path, "drivers", "chromedriver")
                        if os.path.exists(chromedriver_cache):
                            print(f"  → ChromeDriver 캐시 정리 시도 (시도 {attempt})...")
                            # 실행 중인 프로세스 종료
                            kill_chromedriver_processes()
                            time.sleep(2)
                    except:
                        pass
            
            driver_path = ChromeDriverManager().install()
            print(f"  ✓ ChromeDriverManager로 설치 완료: {driver_path}")
            return driver_path
        except PermissionError as e:
            print(f"  ⚠ 권한 오류 발생 (시도 {attempt}/{max_retries}): {e}")
            if attempt < max_retries:
                print(f"  → {attempt + 1}초 후 재시도...")
                kill_chromedriver_processes()
                time.sleep(attempt + 1)
            else:
                raise Exception("ChromeDriver 설치 실패: 권한 문제가 지속됩니다. ChromeDriver 프로세스를 수동으로 종료하거나 관리자 권한으로 실행해주세요.")
        except Exception as e:
            print(f"  ⚠ 오류 발생 (시도 {attempt}/{max_retries}): {e}")
            if attempt < max_retries:
                time.sleep(1)
            else:
                raise

print("ChromeDriver 설정 중...")
driver_path = setup_chromedriver()

# 드라이버 초기화
service = Service(driver_path)
# 서비스 타임아웃 설정 (기본값보다 길게)
service.service_args = ['--timeout=120']  # 2분 타임아웃

driver = webdriver.Chrome(service=service, options=options)
driver.maximize_window()
driver.set_page_load_timeout(30)  # 페이지 로드 타임아웃 30초 설정
driver.implicitly_wait(5)  # 암묵적 대기 시간 설정
wait = WebDriverWait(driver, 10)  # 최대 10초까지 기다림

# User-Agent 설정으로 봇 감지 회피
driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")

# 대학 기관 로그인 - credentials.json에서 읽기
with open('credentials.json', 'r', encoding='utf-8') as f:
    credentials = json.load(f)
    
univ_url = "https://lib.gachon.ac.kr/login"
univ_id = credentials['univ_id']
univ_pw = credentials['univ_pw']

print("=" * 50)
print("기관 로그인 진행 중...")
print("=" * 50)

# 로그인 페이지 접속
print(f"로그인 페이지 접속 중: {univ_url}")
try:
    driver.get(univ_url)
    print("  ✓ 페이지 접속 완료")
except TimeoutException:
    print("  ⚠ 페이지 로드 타임아웃 발생, JavaScript로 강제 이동 시도...")
    try:
        driver.execute_script(f"window.location.href = '{univ_url}';")
        time.sleep(3)
        print("  ✓ JavaScript로 페이지 이동 완료")
    except Exception as e:
        print(f"  ✗ JavaScript 이동 실패: {e}")
        raise

# 페이지 로드 완료 대기 (eager 전략이므로 DOMContentLoaded까지만 대기)
try:
    WebDriverWait(driver, 15).until(
        lambda d: d.execute_script("return document.readyState") in ["complete", "interactive"]
    )
    print("  ✓ 페이지 준비 완료")
    time.sleep(2)  # 추가 안정화 대기
except TimeoutException:
    print("  ⚠ 페이지 로드 타임아웃, 현재 상태로 계속 진행합니다.")
    time.sleep(2)  # 최소한의 대기

try:
    # ID 입력 필드 찾기 (여러 선택자 시도)
    print("ID 입력 필드 찾는 중...")
    id_input = None
    id_selectors = [
        (By.ID, "id"),
        (By.NAME, "id"),
        (By.CSS_SELECTOR, "input[type='text'][name='id']"),
        (By.CSS_SELECTOR, "input[type='text'][id='id']"),
        (By.CSS_SELECTOR, "input[placeholder*='아이디']"),
    ]
    
    for idx, (by, selector) in enumerate(id_selectors, 1):
        try:
            print(f"  시도 {idx}/{len(id_selectors)}: {selector}")
            id_input = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((by, selector))
            )
            if id_input and id_input.is_displayed():
                print(f"  ✓ ID 입력 필드 발견: {selector}")
                break
        except (TimeoutException, NoSuchElementException) as e:
            print(f"  ✗ 실패: {selector}")
            continue
        except Exception as e:
            print(f"  ✗ 오류: {e}")
            continue
    
    if not id_input:
        # 디버깅: 페이지 소스 일부 출력
        print("\n⚠ 디버깅 정보:")
        print(f"현재 URL: {driver.current_url}")
        print(f"페이지 제목: {driver.title}")
        try:
            inputs = driver.find_elements(By.TAG_NAME, "input")
            print(f"발견된 input 요소 수: {len(inputs)}")
            for i, inp in enumerate(inputs[:5]):  # 처음 5개만 출력
                print(f"  Input {i+1}: type={inp.get_attribute('type')}, id={inp.get_attribute('id')}, name={inp.get_attribute('name')}")
        except:
            pass
        raise Exception("ID 입력 필드를 찾을 수 없습니다.")
    
    id_input.clear()
    id_input.send_keys(univ_id)
    print(f"  ✓ ID 입력 완료: {univ_id}")
    
    # 비밀번호 입력 필드 찾기 (여러 선택자 시도)
    print("비밀번호 입력 필드 찾는 중...")
    pw_input = None
    pw_selectors = [
        (By.ID, "password"),
        (By.NAME, "password"),
        (By.CSS_SELECTOR, "input[type='password'][name='password']"),
        (By.CSS_SELECTOR, "input[type='password'][id='password']"),
        (By.CSS_SELECTOR, "input[type='password'][placeholder*='비밀번호']"),
    ]
    
    for idx, (by, selector) in enumerate(pw_selectors, 1):
        try:
            print(f"  시도 {idx}/{len(pw_selectors)}: {selector}")
            pw_input = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((by, selector))
            )
            if pw_input and pw_input.is_displayed():
                print(f"  ✓ 비밀번호 입력 필드 발견: {selector}")
                break
        except (TimeoutException, NoSuchElementException) as e:
            print(f"  ✗ 실패: {selector}")
            continue
        except Exception as e:
            print(f"  ✗ 오류: {e}")
            continue
    
    if not pw_input:
        # 디버깅: 페이지 소스 일부 출력
        print("\n⚠ 디버깅 정보:")
        try:
            password_inputs = driver.find_elements(By.CSS_SELECTOR, "input[type='password']")
            print(f"발견된 password input 요소 수: {len(password_inputs)}")
            for i, inp in enumerate(password_inputs):
                print(f"  Password Input {i+1}: id={inp.get_attribute('id')}, name={inp.get_attribute('name')}")
        except:
            pass
        raise Exception("비밀번호 입력 필드를 찾을 수 없습니다.")
    
    pw_input.clear()
    pw_input.send_keys(univ_pw)
    print("  ✓ 비밀번호 입력 완료")
    
    # 로그인 버튼 찾기 및 클릭
    print("로그인 버튼 찾는 중...")
    login_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']")))
    driver.execute_script("arguments[0].click();", login_button)
    print("  ✓ 로그인 버튼 클릭 완료")
    
    # 로그인 완료 대기
    wait.until(lambda d: "login" not in d.current_url.lower() or d.current_url != univ_url)
    print("\n✅ 로그인 완료!")
    print(f"현재 URL: {driver.current_url}")
    
except Exception as e:
    print(f"❌ 로그인 중 오류 발생: {e}")
    print("로그인을 수동으로 진행해주세요.")

# Advanced Search 페이지 접속
print("\n" + "=" * 50)
print("Advanced Search 페이지 접속 중...")
print("=" * 50)
url = 'https://ieeexplore-ieee-org-ssl.glibproxy.gachon.ac.kr/search/advanced'

# 재시도 로직으로 페이지 접속
max_retries = 5
for attempt in range(1, max_retries + 1):
    try:
        print(f"페이지 접속 시도 {attempt}/{max_retries}: {url}")
        driver.get(url)
        print(f"  ✓ 페이지 접속 완료 (시도 {attempt})")
        break
    except TimeoutException:
        print(f"  ⚠ 페이지 로드 타임아웃 (시도 {attempt}/{max_retries})")
        if attempt < max_retries:
            print("  → JavaScript로 강제 이동 시도...")
            try:
                driver.execute_script(f"window.location.href = '{url}';")
                time.sleep(3)
                print("  ✓ JavaScript로 페이지 이동 완료")
                break
            except Exception as e:
                print(f"  ✗ JavaScript 이동 실패: {e}")
                if attempt < max_retries:
                    print(f"  → {attempt + 1}초 후 재시도...")
                    time.sleep(attempt + 1)
                else:
                    raise
        else:
            print("  ✗ 모든 접속 시도 실패")
            # 마지막 시도: JavaScript로 이동
            try:
                driver.execute_script(f"window.location.href = '{url}';")
                time.sleep(5)
                print("  ✓ 마지막 시도: JavaScript로 페이지 이동 완료")
            except Exception as e:
                print(f"  ✗ 최종 시도 실패: {e}")
                raise

time.sleep(2)  # 초기 로드 대기

# redirect를 위한 F5 새로고침 (여러 번 시도)
for refresh_attempt in range(1, 4):
    try:
        print(f"  → F5 키로 페이지 새로고침 (시도 {refresh_attempt}/3)...")
        body = driver.find_element(By.TAG_NAME, "body")
        body.send_keys(Keys.F5)
        time.sleep(3)  # 새로고침 후 대기
        
        # 페이지가 로드되었는지 확인
        WebDriverWait(driver, 10).until(
            lambda d: d.execute_script("return document.readyState") in ["complete", "interactive"]
        )
        print("  ✓ 새로고침 완료 및 페이지 준비됨")
        break
    except Exception as e:
        print(f"  ⚠ 새로고침 시도 {refresh_attempt} 실패: {e}")
        if refresh_attempt < 3:
            time.sleep(2)
        else:
            print("  ⚠ 새로고침 실패했지만 계속 진행합니다.")

# 페이지 로드 대기 (더보기 버튼 또는 검색 필드가 나타날 때까지)
try:
    wait.until(EC.any_of(
        EC.presence_of_element_located((By.CSS_SELECTOR, "i.fa-plus")),
        EC.presence_of_element_located((By.CSS_SELECTOR, "input[aria-label='Search Term']"))
    ))
    print("  ✓ Advanced Search 페이지 로드 완료")
except:
    print("  ⚠ 페이지 요소 찾기 실패, 현재 상태로 계속 진행합니다.")


# Publication Year 설정 및 검색 버튼 클릭
print("\n" + "=" * 50)
print("Publication Year 설정 중...")
print("=" * 50)

# Publication Year 라벨 클릭 (for="id_1")
print("Publication Year 라벨 찾는 중...")
pub_year_label = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "label[for='id_1']")))
driver.execute_script("arguments[0].scrollIntoView({behavior: 'instant', block: 'center'});", pub_year_label)
driver.execute_script("arguments[0].click();", pub_year_label)
print("  ✓ Publication Year 라벨 클릭 완료")

# 시작 입력 필드에 입력
print("시작 연도 입력 필드 찾는 중...")
year_input = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "input[aria-label='Enter start year of range']")))
driver.execute_script("arguments[0].scrollIntoView({behavior: 'instant', block: 'center'});", year_input)
year_input.click()
year_input.clear()
year_input.send_keys(SET_YEAR)
print("  ✓ 시작 연도 입력 완료")

# 마지막 입력 필드에 입력
print("마지막 연도 입력 필드 찾는 중...")
year_input = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "input[aria-label='Enter end year of range']")))
driver.execute_script("arguments[0].scrollIntoView({behavior: 'instant', block: 'center'});", year_input)
year_input.click()
year_input.clear()
year_input.send_keys(SET_YEAR)
print("  ✓ 마지막 연도 입력 완료")

# 검색 버튼 클릭
print("검색 버튼 찾는 중...")
search_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button.xpl-btn-primary.stats-Adv_search")))
driver.execute_script("arguments[0].scrollIntoView({behavior: 'instant', block: 'center'});", search_button)
driver.execute_script("arguments[0].click();", search_button)
print("  ✓ 검색 버튼 클릭 완료")

print("\n✅ 검색 실행 완료!")
print(f"현재 URL: {driver.current_url}")

print("\n페이지 준비 완료. 이제 검색 버튼을 클릭하거나 추가 작업을 진행하세요.")

ChromeDriver 설정 중...
  ✓ chromedriver_autoinstaller로 설치 완료: C:\Users\gnewsoft\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\chromedriver_autoinstaller\144\chromedriver.exe
기관 로그인 진행 중...
로그인 페이지 접속 중: https://lib.gachon.ac.kr/login
  ✓ 페이지 접속 완료
  ✓ 페이지 준비 완료
ID 입력 필드 찾는 중...
  시도 1/5: id
  ✓ ID 입력 필드 발견: id
  ✓ ID 입력 완료: westhi320
비밀번호 입력 필드 찾는 중...
  시도 1/5: password
  ✓ 비밀번호 입력 필드 발견: password
  ✓ 비밀번호 입력 완료
로그인 버튼 찾는 중...
  ✓ 로그인 버튼 클릭 완료

✅ 로그인 완료!
현재 URL: https://lib.gachon.ac.kr/

Advanced Search 페이지 접속 중...
페이지 접속 시도 1/5: https://ieeexplore-ieee-org-ssl.glibproxy.gachon.ac.kr/search/advanced
  ✓ 페이지 접속 완료 (시도 1)
  → F5 키로 페이지 새로고침 (시도 1/3)...
  ✓ 새로고침 완료 및 페이지 준비됨
  ✓ Advanced Search 페이지 로드 완료

Publication Year 설정 중...
Publication Year 라벨 찾는 중...
  ✓ Publication Year 라벨 클릭 완료
시작 연도 입력 필드 찾는 중...
  ✓ 시작 연도 입력 완료
마지막 연도 입력 필드 찾는 중...
  ✓ 마지막 연도 입력 완료
검색 버튼 찾는 중...
  ✓ 검색 버튼 클릭 완료

✅ 검색 실행 완료!
현재 URL: ht

# IEEE 논문 크롤링

In [5]:
from selenium.webdriver.chrome.webdriver import WebDriver


print("\n" + "=" * 50)
print(f"Publication Title 필터 적용: {TITLE}")
print("=" * 50)


# ===== Seat Limit 감지 및 회피 함수 =====
def random_delay(min_sec=None, max_sec=None):
    """랜덤한 시간만큼 대기 (봇 감지 회피)"""
    min_sec = min_sec or MIN_RANDOM_DELAY
    max_sec = max_sec or MAX_RANDOM_DELAY
    delay = random.uniform(min_sec, max_sec)
    time.sleep(delay)
    return delay


def check_seat_limit(driver):
    """페이지에서 Seat Limit 메시지를 감지한다."""
    seat_limit_keywords = [
        "reached the limit on seats",
        "automatically paused access",
        "has automatically paused",
        "limit on seats for your organization",
        "please try again shortly"
    ]
    
    try:
        page_source = driver.page_source.lower()
        for keyword in seat_limit_keywords:
            if keyword.lower() in page_source:
                return True
    except Exception as e:
        print(f"  ⚠ Seat limit 체크 중 오류: {e}")
    
    return False


def wait_and_retry_on_seat_limit(driver, action_func, action_name="작업", max_retries=None):
    """
    Seat limit이 감지되면 대기 후 재시도하는 래퍼 함수.
    action_func: 실행할 함수 (람다 또는 함수 참조)
    """
    max_retries = max_retries or MAX_SEAT_LIMIT_RETRIES
    
    for attempt in range(1, max_retries + 1):
        # 먼저 seat limit 체크
        if check_seat_limit(driver):
            print(f"\n⚠️ Seat Limit 감지! (시도 {attempt}/{max_retries})")
            print(f"  → {SEAT_LIMIT_WAIT_SECONDS}초({SEAT_LIMIT_WAIT_SECONDS//60}분) 대기 후 재시도합니다...")
            
            # 프로그레스 표시와 함께 대기
            for remaining in range(SEAT_LIMIT_WAIT_SECONDS, 0, -30):
                print(f"  ⏳ 남은 시간: {remaining}초...")
                time.sleep(min(30, remaining))
            
            # 페이지 새로고침
            print("  → 페이지 새로고침 중...")
            try:
                driver.refresh()
                time.sleep(5)
                
                # 새로고침 후에도 seat limit 체크
                if check_seat_limit(driver):
                    print(f"  ⚠ 새로고침 후에도 Seat Limit 유지됨")
                    continue
                else:
                    print(f"  ✓ Seat Limit 해제됨, 작업 재개")
            except Exception as e:
                print(f"  ⚠ 새로고침 실패: {e}")
                continue
        
        # 작업 실행 시도
        try:
            result = action_func()
            return result
        except Exception as e:
            # 작업 실패 시 seat limit 다시 체크
            if check_seat_limit(driver):
                print(f"\n⚠️ {action_name} 중 Seat Limit 감지! (시도 {attempt}/{max_retries})")
                if attempt < max_retries:
                    print(f"  → {SEAT_LIMIT_WAIT_SECONDS}초 대기 후 재시도...")
                    time.sleep(SEAT_LIMIT_WAIT_SECONDS)
                    driver.refresh()
                    time.sleep(5)
                    continue
            else:
                # Seat limit이 아닌 다른 오류
                raise e
    
    raise Exception(f"{action_name} 실패: {max_retries}회 시도 후에도 Seat Limit 해제되지 않음")


def apply_publication_title_filter():
    """Publication Title 필터에서 목표 저널만 선택해 적용한다."""
    
    # Seat limit 체크
    if check_seat_limit(driver):
        print("  ⚠️ Seat Limit 감지됨, 대기 중...")
        time.sleep(SEAT_LIMIT_WAIT_SECONDS)
        driver.refresh()
        time.sleep(5)

    try:
        filter_toggle = WebDriverWait(driver, 15).until(
            EC.element_to_be_clickable((By.XPATH, "//span[contains(@class, 'refinement-title') and text()='Publication Title']"))
        )
        driver.execute_script("arguments[0].scrollIntoView({behavior: 'instant', block: 'center'});", filter_toggle)
        driver.execute_script("arguments[0].click();", filter_toggle)
        print("  ✓ Publication Title 필터 열기 완료")
        random_delay(1, 2)  # 랜덤 지연 추가
    except TimeoutException:
        print("  ⚠ Publication Title 필터를 찾지 못했습니다. 이미 열려 있을 수 있습니다.")

    # 검색 창에 TITLE 입력 (필터 내부 검색 기능)
    try:
        search_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "input[type='search'][aria-label='Enter search text'][placeholder='Enter Title']"))
        )
        driver.execute_script("arguments[0].scrollIntoView({behavior: 'instant', block: 'center'});", search_input)
        search_input.clear()
        search_input.send_keys(TITLE)
        print(f"  ✓ 검색 창에 '{TITLE}' 입력 완료")
        random_delay(1, 2)  # 랜덤 지연 추가
    except TimeoutException:
        print("  ⚠ 검색 창을 찾지 못했습니다. 필터 목록에서 직접 찾습니다.")

    label = WebDriverWait(driver, 15).until(
        EC.element_to_be_clickable((
            By.XPATH,
            f"//span[contains(@class, 'refinement-label') and normalize-space()='{TITLE}']"
        ))
    )
    driver.execute_script("arguments[0].scrollIntoView({behavior: 'instant', block: 'center'});", label)
    driver.execute_script("arguments[0].click();", label)
    print(f"  ✓ '{TITLE}' 체크박스 선택 완료")

    apply_button = WebDriverWait(driver, 15).until(
        EC.element_to_be_clickable((By.ID, "Publication Title-apply-btn"))
    )
    driver.execute_script("arguments[0].click();", apply_button)
    print("  ✓ Publication Title 필터 적용 완료")
    WebDriverWait(driver, 30).until(
        lambda d: d.execute_script("return document.readyState") == "complete"
    )
    random_delay(2, 4)  # 랜덤 지연 추가

def navigate_to_target_page(target_page: int, current_page: int = 1):
    """Next 버튼을 계속 누르면서 원하는 페이지가 나타나면 멈춘 다음 그 페이지로 이동한다."""
    if target_page < 1:
        raise ValueError("target_page는 1 이상의 정수여야 합니다.")
    
    if target_page == current_page:
        print(f"  ✓ 이미 {target_page} 페이지에 있습니다.")
        return target_page
    
    print(f"\n▶ START_PAGE={target_page}로 이동 준비...")
    print(f"  현재 페이지: {current_page}, 목표 페이지: {target_page}")
    
    # 현재 페이지 세트 계산 (10개씩 묶음)
    target_block = (target_page - 1) // 10
    
    # 페이지 하단으로 스크롤
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    random_delay(1, 2)  # 랜덤 지연 추가
    
    # 목표 페이지가 포함된 세트까지 Next 버튼 클릭
    current_block = (current_page - 1) // 10
    while current_block < target_block:
        # Seat limit 체크
        if check_seat_limit(driver):
            print("  ⚠️ Seat Limit 감지됨, 대기 중...")
            time.sleep(SEAT_LIMIT_WAIT_SECONDS)
            driver.refresh()
            time.sleep(5)
            
        try:
            # Next 버튼 찾기
            next_block_button = WebDriverWait(driver, 8).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "button[aria-label='Next 10 pages of search results']"))
            )
            driver.execute_script("arguments[0].scrollIntoView({behavior: 'instant', block: 'center'});", next_block_button)
            driver.execute_script("arguments[0].click();", next_block_button)
            current_block += 1
            print(f"  → Next 버튼 클릭 ({current_block * 10 - 9}~{current_block * 10} 페이지 세트로 이동)")
            time.sleep(PAGE_CHANGE_DELAY)
            random_delay()  # 추가 랜덤 지연
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            random_delay(1, 2)
        except TimeoutException:
            print("  ⚠ Next 버튼을 찾지 못했습니다. 마지막 페이지 세트로 판단합니다.")
            raise RuntimeError(f"요청한 START_PAGE={target_page}에 도달하기 전에 페이지가 끝났습니다.")
    
    # 목표 페이지 버튼 찾기 및 클릭
    try:
        page_button = locate_page_button(target_page)
        driver.execute_script("arguments[0].scrollIntoView({behavior: 'instant', block: 'center'});", page_button)
        driver.execute_script("arguments[0].click();", page_button)
        print(f"  → {target_page} 페이지 이동, {PAGE_CHANGE_DELAY}초 대기")
        time.sleep(PAGE_CHANGE_DELAY)
        random_delay()  # 추가 랜덤 지연
        
        WebDriverWait(driver, 20).until(lambda d: d.execute_script("return document.readyState") == "complete")
        driver.execute_script("window.scrollTo(0, 0);")
        random_delay(1, 2)
        
        print(f"▶ {target_page} 페이지에서 다운로드를 재개합니다.")
        return target_page
    except TimeoutException:
        raise RuntimeError(f"{target_page} 페이지 버튼을 찾을 수 없습니다. 페이지 이동 실패")

def select_all_results():
    """현재 페이지에서 결과 전체 선택 체크박스를 클릭한다."""
    # Seat limit 체크
    if check_seat_limit(driver):
        print("  ⚠️ Seat Limit 감지됨, 대기 중...")
        time.sleep(SEAT_LIMIT_WAIT_SECONDS)
        driver.refresh()
        time.sleep(5)
    
    checkbox = WebDriverWait(driver, 15).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, "input.results-actions-selectall-checkbox"))
    )
    driver.execute_script(
        "arguments[0].scrollIntoView({behavior: 'instant', block: 'center'});",
        checkbox,
    )
    if not checkbox.is_selected():
        driver.execute_script("arguments[0].click();", checkbox)
    print("  ✓ 전체 결과 선택 완료")
    # 전체 선택 후 스크롤을 최상단으로 이동 (Download PDFs 버튼이 상단에 있음)
    driver.execute_script("window.scrollTo(0, 0);")
    random_delay(1, 2)  # 랜덤 지연 추가
    # Download PDFs 버튼이 나타날 때까지 대기
    random_delay(1, 2)  # 랜덤 지연 추가


def set_items_per_page_10():
    """Items Per Page를 10으로 설정한다 (가능할 때만)."""
    try:
        per_page_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, "dropdownPerPageLabel"))
        )
        driver.execute_script(
            "arguments[0].scrollIntoView({behavior: 'instant', block: 'center'});",
            per_page_button,
        )
        driver.execute_script("arguments[0].click();", per_page_button)
        print("  ✓ Items Per Page 드롭다운 버튼 클릭")
    except TimeoutException:
        print("  ⚠ Items Per Page 드롭다운 버튼을 찾지 못했습니다. (기본 값으로 진행)")
        return

    try:
        option_10 = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable(
                (
                    By.XPATH,
                    "//button[contains(@class, 'dropdown-item') and contains(@class, 'filter-popover-option') and normalize-space()='10']",
                )
            )
        )
        driver.execute_script("arguments[0].click();", option_10)
        print("  ✓ Items Per Page = 10 선택 완료")
        random_delay(2, 4)  # 랜덤 지연 추가
    except TimeoutException:
        print("  ⚠ '10' 옵션을 찾지 못했습니다. 현재 설정 값으로 계속 진행합니다.")


def trigger_download():
    """현재 페이지에서 선택된 논문을 일괄 다운로드한다."""
    
    # Seat limit 체크
    if check_seat_limit(driver):
        print("  ⚠️ Seat Limit 감지됨, 대기 중...")
        time.sleep(SEAT_LIMIT_WAIT_SECONDS)
        driver.refresh()
        time.sleep(5)
        return False  # 다시 시도하도록 False 반환
    
    # 스크롤을 최상단으로 이동 (Download PDFs 버튼이 상단에 있음)
    driver.execute_script("window.scrollTo(0, 0);")
    random_delay(0.5, 1)  # 랜덤 지연 추가

    # 1) Download PDFs 버튼 클릭 (여러 선택자 시도)
    primary_button = None
    download_button_selectors = [
        # 클래스 기반 선택자 (가장 확실함)
        (By.CSS_SELECTOR, "button.xpl-btn-primary"),
        (By.XPATH, "//button[contains(@class, 'xpl-btn-primary')]"),
        # 텍스트 기반 선택자
        (By.XPATH, "//button[contains(normalize-space(text()), 'Download PDFs')]"),
        (By.XPATH, "//button[contains(text(), 'Download PDFs')]"),
        (By.XPATH, "//button[contains(., 'Download PDFs')]"),
        # 클래스와 텍스트 조합
        (By.XPATH, "//button[contains(@class, 'xpl-btn-primary') and contains(., 'Download PDFs')]"),
        (By.XPATH, "//button[contains(@class, 'xpl-btn-primary') and contains(., 'PDFs')]"),
        # aria-label 기반 (혹시 모를 경우)
        (By.CSS_SELECTOR, "button[aria-label*='Download PDFs']"),
        (By.CSS_SELECTOR, "button[aria-label*='download PDFs']"),
    ]
    
    print("  🔍 Download PDFs 버튼 찾는 중...")
    for idx, (by, value) in enumerate(download_button_selectors):
        try:
            primary_button = WebDriverWait(driver, 3).until(
                EC.presence_of_element_located((by, value))
            )
            # 버튼이 실제로 보이고 활성화되어 있는지 확인
            if primary_button and primary_button.is_displayed():
                # 클릭 가능할 때까지 추가 대기
                try:
                    WebDriverWait(driver, 2).until(EC.element_to_be_clickable(primary_button))
                except:
                    pass
                if primary_button.is_enabled():
                    print(f"  ✓ Download PDFs 버튼 발견 (선택자 {idx+1}/{len(download_button_selectors)}: {value})")
                    break
            primary_button = None
        except (TimeoutException, NoSuchElementException):
            continue
        except Exception as e:
            continue
    
    # 최종 확인: 모든 버튼을 찾아서 텍스트로 확인
    if primary_button is None:
        print("  ⚠ 기본 선택자로 버튼을 찾지 못했습니다. 모든 버튼 검색 중...")
        try:
            all_buttons = driver.find_elements(By.TAG_NAME, "button")
            for btn in all_buttons:
                if btn.is_displayed():
                    btn_text = btn.text.strip()
                    if "Download PDFs" in btn_text or "download pdfs" in btn_text.lower():
                        primary_button = btn
                        print(f"  ✓ 텍스트로 버튼 발견: '{btn_text}'")
                        break
        except Exception as e:
            print(f"  ⚠ 버튼 검색 중 오류: {e}")
    
    if primary_button is None:
        # 디버깅 정보 출력
        try:
            all_buttons = driver.find_elements(By.CSS_SELECTOR, "button.xpl-btn-primary")
            print(f"  ⚠ xpl-btn-primary 클래스를 가진 버튼 수: {len(all_buttons)}")
            for i, btn in enumerate(all_buttons):
                print(f"    버튼 {i+1}: 텍스트='{btn.text.strip()}', 표시={btn.is_displayed()}, 활성화={btn.is_enabled()}")
        except:
            pass
        raise TimeoutException("Download PDFs 버튼을 찾을 수 없습니다. 페이지 상태를 확인해주세요.")
    
    driver.execute_script(
        "arguments[0].scrollIntoView({behavior: 'instant', block: 'center'});",
        primary_button,
    )
    # 버튼이 완전히 활성화될 때까지 추가 대기
    random_delay(0.5, 1)  # 랜덤 지연 추가
    driver.execute_script("arguments[0].click();", primary_button)
    print("  ✓ Download PDFs 버튼 클릭")

    # 2) 모달 내부 Download 버튼 클릭
    modal_button = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable(
            (By.CSS_SELECTOR, "button.stats-SearchResults_BulkPDFDownload")
        )
    )
    random_delay(1, 2)  # 모달 안정화를 위한 랜덤 지연
    driver.execute_script("arguments[0].click();", modal_button)
    print("  ✓ 모달 내부 Download 버튼 클릭")

    # 3) 다운로드 대기 및 모달 닫기
    print(f"  ⏳ PDF 다운로드 대기 중 ({DOWNLOAD_WAIT_SECONDS}초)...")
    start_time = time.time()
    close_icon = None
    
    # 다양한 닫기 버튼 선택자 (우선순위 순)
    close_selectors = [
        (By.CSS_SELECTOR, "i.icon.modal-close"),                    # 기존: icon modal-close 클래스
        (By.CSS_SELECTOR, "i.modal-close"),                         # modal-close 클래스만
        (By.CSS_SELECTOR, "i[class*='modal-close']"),               # class 속성에 modal-close 포함
        (By.XPATH, "//i[contains(@class, 'modal-close')]"),         # XPath로 modal-close 포함
        (By.CSS_SELECTOR, "i.fas.fa-times.icon-size-md"),           # 기존: Font Awesome X 아이콘
        (By.CSS_SELECTOR, "i.fa-times"),                            # fa-times 클래스
        (By.XPATH, "//i[contains(@class, 'fa-times')]"),            # XPath로 fa-times 포함
        (By.CSS_SELECTOR, "button[aria-label*='close']"),          # aria-label에 close 포함
        (By.CSS_SELECTOR, "button[aria-label*='Close']"),          # aria-label에 Close 포함
        (By.XPATH, "//button[contains(@aria-label, 'close') or contains(@aria-label, 'Close')]"),  # XPath로 aria-label 확인
    ]

    # 각 선택자를 순차적으로 시도
    for idx, (by, value) in enumerate(close_selectors):
        try:
            close_icon = WebDriverWait(driver, DOWNLOAD_WAIT_SECONDS).until(
                EC.element_to_be_clickable((by, value))
            )
            if close_icon and close_icon.is_displayed():
                print(f"  ✓ 닫기 버튼 발견 (선택자 {idx+1}/{len(close_selectors)}: {value})")
                break
        except TimeoutException:
            continue
        except Exception as e:
            # 다른 예외도 무시하고 다음 선택자 시도
            continue

    if close_icon:
        try:
            driver.execute_script("arguments[0].scrollIntoView({behavior: 'instant', block: 'center'});", close_icon)
            driver.execute_script("arguments[0].click();", close_icon)
            elapsed = time.time() - start_time
            print(f"  ✓ 다운로드 모달 닫기 완료 (대기 {elapsed:.1f}초)")
        except Exception as e:
            print(f"  ⚠ 닫기 버튼 클릭 실패: {e}")
            # 대체 방법 시도
            try:
                driver.execute_script("arguments[0].click();", close_icon)
            except:
                pass
    else:
        print(
            f"  ⚠ {DOWNLOAD_WAIT_SECONDS}초 내 닫기 버튼이 나타나지 않았습니다. 수동 확인이 필요할 수 있습니다."
        )
        # 모든 선택자로 다시 한 번 찾기 시도 (더 짧은 타임아웃)
        for by, value in close_selectors:
            try:
                elements = driver.find_elements(by, value)
                for elem in elements:
                    if elem.is_displayed() and elem.is_enabled():
                        close_icon = elem
                        driver.execute_script("arguments[0].click();", close_icon)
                        print(f"  ✓ 닫기 버튼 발견 및 클릭 (대체 방법: {value})")
                        break
                if close_icon:
                    break
            except:
                continue
        
        if not close_icon:
            remaining = max(DOWNLOAD_WAIT_SECONDS - (time.time() - start_time), 0)
            if remaining > 0:
                time.sleep(min(remaining, 10))  # 최대 10초만 추가 대기
            print("  ⚠ 닫기 아이콘 미감지 상태로 다음 단계로 진행합니다.")

    # 4) 페이지 하단까지 스크롤 (다음 페이지 이동 전 안정화용)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    random_delay(1, 2)  # 랜덤 지연 추가
    
    return True  # 성공


def process_current_page(page_number: int):
    """현재 페이지 처리 (Seat limit 대응 포함)"""
    print(f"\n--- {page_number} 페이지 처리 시작 ---")
    
    max_retries = MAX_SEAT_LIMIT_RETRIES
    for attempt in range(1, max_retries + 1):
        # Seat limit 체크
        if check_seat_limit(driver):
            print(f"\n⚠️ Seat Limit 감지! (시도 {attempt}/{max_retries})")
            print(f"  → {SEAT_LIMIT_WAIT_SECONDS}초({SEAT_LIMIT_WAIT_SECONDS//60}분) 대기 후 재시도합니다...")
            
            # 프로그레스 표시와 함께 대기
            for remaining in range(SEAT_LIMIT_WAIT_SECONDS, 0, -60):
                print(f"  ⏳ 남은 시간: {remaining}초...")
                time.sleep(min(60, remaining))
            
            # 페이지 새로고침
            print("  → 페이지 새로고침 중...")
            driver.refresh()
            time.sleep(5)
            random_delay(2, 5)
            continue
        
        try:
            # 1) 현재 페이지 전체 선택
            select_all_results()
            
            # 2) 선택된 논문 일괄 다운로드
            download_result = trigger_download()
            
            if download_result:
                print(f"--- {page_number} 페이지 처리 완료 ---")
                # 다운로드 후 추가 랜덤 대기 (Seat limit 회피)
                delay = random_delay(5, 15)
                print(f"  → 다음 페이지 전 {delay:.1f}초 대기 (Seat limit 회피)")
                return True
            else:
                print(f"  ⚠ 다운로드 실패, 재시도 중...")
                continue
                
        except Exception as e:
            if check_seat_limit(driver):
                print(f"  ⚠️ 작업 중 Seat Limit 감지됨")
                continue
            else:
                raise e
    
    print(f"  ❌ {page_number} 페이지 처리 실패: {max_retries}회 재시도 후에도 Seat Limit 해제되지 않음")
    return False


def locate_page_button(page_number: int, timeout: int = 10):
    selectors = [
        (By.CSS_SELECTOR, f"button.stats-Pagination_{page_number}"),
        (By.CSS_SELECTOR, f"button[aria-label='Page {page_number} of search results']"),
        (By.XPATH, f"//button[contains(@class, 'stats-Pagination') and normalize-space()='{page_number}']")
    ]
    end_time = time.time() + timeout
    while time.time() < end_time:
        for by, value in selectors:
            elements = driver.find_elements(by, value)
            for element in elements:
                if element.is_displayed() and element.is_enabled():
                    return element
        time.sleep(0.2)
    raise TimeoutException(f"{page_number} 페이지 버튼을 찾지 못했습니다.")


def go_to_next_page(current_page: int):
    """다음 페이지로 이동 (Seat limit 대응 포함)"""
    
    # Seat limit 체크
    if check_seat_limit(driver):
        print("  ⚠️ Seat Limit 감지됨, 대기 중...")
        time.sleep(SEAT_LIMIT_WAIT_SECONDS)
        driver.refresh()
        time.sleep(5)
    
    next_page = current_page + 1
    current_block = (current_page - 1) // 10
    target_block = (next_page - 1) // 10

    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    random_delay(1, 2)  # 랜덤 지연 추가

    while current_block < target_block:
        # Seat limit 체크
        if check_seat_limit(driver):
            print("  ⚠️ Seat Limit 감지됨, 대기 중...")
            time.sleep(SEAT_LIMIT_WAIT_SECONDS)
            driver.refresh()
            time.sleep(5)
            
        try:
            next_block_button = WebDriverWait(driver, 8).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "button[aria-label='Next 10 pages of search results']"))
            )
            driver.execute_script("arguments[0].click();", next_block_button)
            current_block += 1
            print("  → Next 버튼 클릭, 페이지 세트 전환 후 대기")
            time.sleep(PAGE_CHANGE_DELAY)
            random_delay()  # 추가 랜덤 지연
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            random_delay(1, 2)
        except TimeoutException:
            print("  ⚠ Next 버튼을 찾지 못했습니다. 마지막 페이지 세트로 판단합니다.")
            return None

    try:
        page_button = locate_page_button(next_page)
    except TimeoutException:
        print(f"  ⚠ {next_page} 페이지 버튼을 찾을 수 없습니다. 다음 페이지로 이동 실패")
        return None

    try:
        driver.execute_script("arguments[0].scrollIntoView({behavior: 'instant', block: 'center'});", page_button)
        driver.execute_script("arguments[0].click();", page_button)
    except StaleElementReferenceException:
        page_button = locate_page_button(next_page)
        driver.execute_script("arguments[0].scrollIntoView({behavior: 'instant', block: 'center'});", page_button)
        driver.execute_script("arguments[0].click();", page_button)

    print(f"  → {next_page} 페이지 이동, {PAGE_CHANGE_DELAY}초 대기")
    time.sleep(PAGE_CHANGE_DELAY)
    random_delay()  # 추가 랜덤 지연

    WebDriverWait(driver, 20).until(lambda d: d.execute_script("return document.readyState") == "complete")
    driver.execute_script("window.scrollTo(0, 0);")
    random_delay(1, 2)

    return next_page


apply_publication_title_filter()

# Items Per Page = 10 설정 (페이지 이동 전에 한 번만 수행)
set_items_per_page_10()

current_page = 1
visited_pages = 0

if START_PAGE < 1:
    raise ValueError("START_PAGE는 1 이상의 정수여야 합니다.")

if START_PAGE > current_page:
    current_page = navigate_to_target_page(START_PAGE, current_page)
elif START_PAGE < current_page:
    print(f"START_PAGE={START_PAGE}는 현재 페이지보다 작습니다. 1페이지에서 시작합니다.")
    START_PAGE = 1
    current_page = 1

while visited_pages < MAX_PAGE_VISITS:
    success = process_current_page(current_page)
    
    if not success:
        print(f"\n⚠ {current_page} 페이지 처리 실패. 10분 후 재시도...")
        time.sleep(600)  # 10분 대기
        continue
    
    visited_pages += 1

    next_page_number = go_to_next_page(current_page)
    if next_page_number is None:
        print("\n✅ 더 이상 이동할 페이지가 없어 작업을 종료합니다.")
        break

    current_page = next_page_number
else:
    print(f"\n⚠ 안전장치(MAX_PAGE_VISITS={MAX_PAGE_VISITS})에 도달하여 루프를 종료합니다.")


Publication Title 필터 적용: IEEE Transactions on Geoscience and Remote Sensing
  ✓ Publication Title 필터 열기 완료
  ✓ 검색 창에 'IEEE Transactions on Geoscience and Remote Sensing' 입력 완료
  ✓ 'IEEE Transactions on Geoscience and Remote Sensing' 체크박스 선택 완료
  ✓ Publication Title 필터 적용 완료
  ✓ Items Per Page 드롭다운 버튼 클릭
  ✓ Items Per Page = 10 선택 완료

▶ START_PAGE=232로 이동 준비...
  현재 페이지: 1, 목표 페이지: 232
  → Next 버튼 클릭 (1~10 페이지 세트로 이동)
  → Next 버튼 클릭 (11~20 페이지 세트로 이동)
  → Next 버튼 클릭 (21~30 페이지 세트로 이동)
  → Next 버튼 클릭 (31~40 페이지 세트로 이동)
  → Next 버튼 클릭 (41~50 페이지 세트로 이동)
  → Next 버튼 클릭 (51~60 페이지 세트로 이동)
  → Next 버튼 클릭 (61~70 페이지 세트로 이동)
  → Next 버튼 클릭 (71~80 페이지 세트로 이동)
  → Next 버튼 클릭 (81~90 페이지 세트로 이동)
  → Next 버튼 클릭 (91~100 페이지 세트로 이동)
  → Next 버튼 클릭 (101~110 페이지 세트로 이동)
  → Next 버튼 클릭 (111~120 페이지 세트로 이동)
  → Next 버튼 클릭 (121~130 페이지 세트로 이동)
  → Next 버튼 클릭 (131~140 페이지 세트로 이동)
  → Next 버튼 클릭 (141~150 페이지 세트로 이동)
  → Next 버튼 클릭 (151~160 페이지 세트로 이동)
  → Next 버튼 클릭 (161~170 페이지 세트로 이동)
  → Next 버튼 클릭 (171

TimeoutException: Message: 
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x9c7d13
	0x9c7d54
	0x7eb290
	0x8356ea
	0x83598b
	0x877912
	0x858004
	0x875111
	0x857d56
	0x8294d9
	0x82a294
	0xc3bb64
	0xc37215
	0xc53fad
	0x9e1ef8
	0x9e9b0d
	0x9d0738
	0x9d0902
	0x9ba1da
	0x753f5d49
	0x7796d5db
	0x7796d561
